In [4]:
cd /content/drive/Shareddrives/과학기술일자리진흥원_은채/200901_210228_인턴/지식재산연구원 논문/동향분석(딥러닝)/기업정보연계

/content/drive/Shareddrives/과학기술일자리진흥원_은채/200901_210228_인턴/지식재산연구원 논문/동향분석(딥러닝)/기업정보연계


# KSIC -> IPC

In [7]:
import pandas as pd

- table 정제

In [120]:
table = pd.read_csv('기업정보연계+KSIC_IPC_연계_final.CSV', encoding='cp949')

In [121]:
table

,농림어업,광업,식료품 제조업,음료제조업,담배제조업,섬유제품 제조업\n:의복제외,"의복, 의복 액세서리 및 모피제품 제조업","가죽, 가방 및 신발제조업",목재 및\n나무제품제조업; 가구제외,"펄프, 종이 및\n종이제품 제조업",인쇄 및\n기록매체복제업,"코크스, 연탄 및\n석유정제품 제조업",기초 화학물질제조업,비료 및 질소 화학물 제조업,합성고무 및\n플라스틱 물질 제조업,살충제 및 기타농약제조업,"잉크, 페인트 코팅제 및 유사제품 제조업","세제, 화장품 및 광택제 제조업",그 외 기타 화학제품 제조업,화학섬유 제조업,의료용 물질 및 의약품 제조업,고무제품 및\n플라스틱 제품 제조업,비금속광물제품 제조업,1차 철강제조업,1차 비철금속 제조업,금속주조업,"구조용 금속제품,\n탱크, 증기발생기 제조업",무기 및 총포탄 제조업,기타 금속가공 제품 제조업,반도체 제조업,전자부품 제조업,컴퓨터 및 주변장치 제조업,통신 및 방송 장비 제조업,영상 및 음향기기 제조업,마그네틱 및 광학 매체 제조업,의료용 기기 제조업,"측정, 시험, 향해, 제어, 및 기타 정밀기기 제조업","안경, 사진장비 및\n기타 광학기기 제조업",시계 및 시계부품 제조업,"전동기, 발전기 및\n전기변환, 공급, 제어장치 제조업",일차전지 및 축전지 제조업,절연선 및 케이블제조업,전구 및 조명장치 제조업,가정용 기기 제조업,기타 전기장비 제조업,사무용 기계 및\n장비 제조업,사무용 이외의 일반기계 제조업,특수 기계제조업,자동차 제조업,선박 제조업,철도장비 제조업,항공기 제조업,전투용 차량 제조업,모터사이클 제조업,그 외 분류 안된\n운송장비 제조업,가구 제조업,기타제품 제조업,"전기, 가스, 증기, 수도","하수, 폐기물처리, 원료재생, 환경복원","건설업, 건축기술 서비스","컴퓨터 프로그래밍, 정보서비스업"
0,A01B27,C22B,A21D,C12C,A24B,D04D,A41B,A43B,A44C23,B42F,B41M,C10G,C01C,C05B,C08B,A01N,C09D,A61Q,A62D,D01C,A61P,B29B,A47K,B21C,C22F,B22D,A47H,F41A,A01L,G11C,B05D,B41J2,G03H,G03B31,G11B,A61B,G01B,G02B,G04B,F99Z,H01M,H01B,B60Q,A45D,B60M,B41L,B01B,A01B(A01B27/02 제외),B60B,B60V,B60L13,B64B,F41H7,B62J,B62B,A47B,A41G,F02C,B09B,E01C,G06D
1,A01B02,E21D,A23B,C12F,A24D,D04G,A41C,A43C,B27D,D21C,B42D,C10L,C01D,C05C,C08F,A01P,NaN,C09F,B01J,D01D,C07D,B29C,B24D,C21B,C25C,NaN,B21G,F41B,A44B11,H01L,B81B,B41J27,H01P,G10L,NaN,A61C,G01C,G02C,G04C,H02B,NaN,H01H,F21H,A47G,C25F,B41J1,B01D,A01F,B60D,B63B,B61B,B64C,NaN,B62K11,B62C,A47C,A44B(A44B11 제외),F03D,B09C,E01D,G06E
2,A01C,NaN,A23C,C12G,A24F,D04H,A41D,A45C,B27K,D21H,NaN,NaN,C01F,C05D,C08G,NaN,NaN,C09G,C06B,D01F,C07J,B29D,B28B,C21C,G21H(G21H1 제외),NaN,F16T,F41C,B21D,NaN,B81C,B41J29,H01Q,H03J,NaN,A61D,G01D,G03D,G04D,H02J,NaN,H01R,F21K,A47J,G08B,G03G,B04C,A01J,B60F,B63C,B61C,B64D,NaN,B62M6,B62H,A47D,A44C(A44C23제외),H02S,B65F,E01F,G06F(G06F1 제외)
3,A01D,NaN,A23D,C12H,NaN,D06C,A41F,B68B,B27H,D21J,NaN,NaN,C01G,C05F,C08J,NaN,NaN,C11D,C06C,NaN,C07K,B29K,B28C,C21D,NaN,NaN,F17B,F41F,B21F,NaN,B82B,B41J31,H01S,H04H,NaN,A61F,G01F,G02F2,G04F,H02K,NaN,H02G,F21L,A47L,G08G,G06C,B05B,A01M,B60G,B63G,B61D,B64F,NaN,B62M7,B62K(B62K11 제외),A47F,A45B,G21H1,C02F,E02B,G06F(G06F3 제외)
4,A01G,NaN,A23F,NaN,NaN,D06J,A42B(A42B3 제외),B68C,B27M,NaN,NaN,NaN,C07B,C05G,C08K,NaN,NaN,D06L,C06D,NaN,C12N,B29L,B28D,C22C,NaN,NaN,F22B,F41G,B21H,NaN,B82Y,B41J32,H03B,H04N,NaN,A61G,G01H,G02F3,G04G,H02N,NaN,NaN,F21S,D06F,G10K,G06M,B23F,A21B,B60H,B63H,B61F,B64G,NaN,NaN,B62L,NaN,A45F,NaN,G21F,E02D,G06G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F02F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F16N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F16P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F26B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
table.columns = pd.Series(table.columns).str.replace('\n', ' ')

In [123]:
table['특수 기계제조업']

0     A01B(A01B27/02 제외)
1                   A01F
2                   A01J
3                   A01M
4                   A21B
             ...        
92                  F02F
93                  F16N
94                  F16P
95                  F26B
96                  H05H
Name: 특수 기계제조업, Length: 97, dtype: object

- query 추출

In [124]:
def change_format(x):
    x = x.strip()
    if len(x) == 4:
        x = x + '*'
    else:
        if '/' not in x:
            x = x[:4] + '-' + x[4:].zfill(3) + '*'
        else:
            x2 = x.split('/')
            x = x2[0][:4] + '-' + x2[0][4:].zfill(3) + '/' + x2[1]
    return x

In [125]:
def make_query(i):
    result = []
    exceptions = list(filter(lambda x: '제외' in x,table[table.columns[i]][table[table.columns[i]].notnull()])) 
    # exception 있는 코드 제외하고 나열
    for x in table[table.columns[i]][table[table.columns[i]].notnull()]:
        x = x.strip()
        if x in exceptions:
            continue
        else:
            x = change_format(x)
            result.append(x)
    # exception 있는 경우에 대해
    keys = [change_format(e.split('(')[0]) for e in exceptions]
    exc_dict = {k: [] for k in keys}
    for e in exceptions:
        exc_dict[change_format(e.split('(')[0])].append(change_format(e.split('(')[1].replace(')','').replace('제외','')))
    for k, v in exc_dict.items():
        result.append('({} not ({}))'.format(k, ' or '.join(v)))
    # 최종 query
    return 'IC = ({})'.format(' or '.join(result))

In [128]:
from tqdm import tqdm

queries = {}
for i, column in tqdm(enumerate(table.columns)):
    query = make_query(i)
    queries[column] = query

61it [00:00, 1702.72it/s]


In [129]:
queries

{'1차 비철금속 제조업': 'IC = (C22F* or C25C* or G21H-(G21H1 제외)* or (G21H* not (G21H-001*)))',
 '1차 철강제조업': 'IC = (B21C* or C21B* or C21C* or C21D* or C22C* or F16S* or F17C*)',
 '가구 제조업': 'IC = (A47B* or A47C* or A47D* or A47F*)',
 '가정용 기기 제조업': 'IC = (A45D* or A47G* or A47J* or A47L* or D06F* or E06C* or F24B* or F24C* or H05B*)',
 '가죽, 가방 및 신발제조업': 'IC = (A43B* or A43C* or A45C* or B68B* or B68C*)',
 '건설업, 건축기술 서비스': 'IC = (E01C* or E01D* or E01F* or E02B* or E02D* or E03B* or E03C* or E03F* or E04H* or E05B* or E05C* or E06B* or E21C* or E99Z* or F42D*)',
 '고무제품 및 플라스틱 제품 제조업': 'IC = (B29B* or B29C* or B29D* or B29K* or B29L* or A42B-003* or B60C* or B65D* or B67D* or C08C* or F16L*)',
 '광업': 'IC = (C22B* or E21D*)',
 '구조용 금속제품, 탱크, 증기발생기 제조업': 'IC = (A47H* or B21G* or F16T* or F17B* or F22B* or F22G* or F24D* or F24H* or G21B* or G21C* or G21D*)',
 '그 외 기타 화학제품 제조업': 'IC = (A62D* or B01J* or C06B* or C06C* or C06D* or C06F* or C08H* or C09H* or C09J* or C09K* or C10M* or C10N* or C11B* o